# Genk network - non-linear model

## Imports

In [28]:
import modesto.main
import modesto.utils as ut
from pkg_resources import resource_filename
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

%matplotlib notebook

## Optimization settings

In [29]:
horizon = 5*3600
time_step = 3*60

n_neighs = 2

start_time = pd.Timestamp('20140101')
n_steps = int(horizon/time_step)

## Neighborhood data

In [30]:
neighs = ['WaterscheiGarden', 
          'ZwartbergNEast', 
          'ZwartbergNWest', 
          'ZwartbergSouth', 
          'OudWinterslag', 
          'Winterslag',
          'Boxbergheide', 
          'TermienEast', 
          'TermienWest']

In [31]:
all_pipes = ['dist_pipe{}'.format(i) for i in range(14)]

diams = {1: [350, 0, 350],
         2: [400, 250, 350],
         3: [450, 250, 350, 200, 200],
         4: [500, 250, 350, 350, 200, 300],
         5: [500, 250, 350, 350, 400, 300, 200, 200],
         6: [600, 250, 350, 500, 200, 300, 400, 200, 350, 0, 350],
         7: [700, 250, 350, 600, 200, 300, 500, 200, 500, 400, 350],
         8: [700, 250, 350, 600, 200, 300, 500, 200, 500, 400, 350, 200, 200],
         9: [700, 250, 350, 600, 200, 300, 600, 200, 500, 400, 350, 300, 200, 250]}

diameters = diams[n_neighs]

### Network graph

In [32]:
 g = nx.DiGraph()

g.add_node('Producer', x=5000, y=5000, z=0,
           comps={'plant': 'Plant'})
g.add_node('p1', x=3500, y=6100, z=0,
           comps={})
if n_neighs >= 1:
    g.add_node('WaterscheiGarden', x=3500, y=5100, z=0,
               comps={'building': 'SubstationepsNTU',})
                      # 'DHW': 'BuildingFixed'})
if n_neighs >= 2:
    g.add_node('ZwartbergNEast', x=3300, y=6700, z=0,
               comps={'building': 'SubstationepsNTU',})
                     # 'DHW': 'BuildingFixed'})
if n_neighs >= 3:
    g.add_node('p2', x=1700, y=6300, z=0,
               comps={})
    g.add_node('ZwartbergNWest', x=1500, y=6600, z=0,
               comps={'building': 'SubstationepsNTU',})
                      # 'DHW': 'BuildingFixed'})
if n_neighs >= 4:
    g.add_node('ZwartbergSouth', x=2000, y=6000, z=0,
               comps={'building': 'SubstationepsNTU',})
                      # 'DHW': 'BuildingFixed'})
if n_neighs >= 5:
    g.add_node('p3', x=250, y=5200, z=0,
               comps={})
    g.add_node('OudWinterslag', x=1700, y=4000, z=0,
               comps={'building': 'SubstationepsNTU',})
                      # 'DHW': 'BuildingFixed'})
if n_neighs >= 6:
    g.add_node('p4', x=0, y=2700, z=0,
               comps={})
    g.add_node('Winterslag', x=1000, y=2500, z=0,
               comps={'building': 'SubstationepsNTU',})
                      # 'DHW': 'BuildingFixed'})
if n_neighs >= 7:
    g.add_node('Boxbergheide', x=-1200, y=2100, z=0,
               comps={'building': 'SubstationepsNTU',})
                      # 'DHW': 'BuildingFixed'})
if n_neighs >= 8:
    g.add_node('p5', x=620, y=700, z=0,
               comps={})
    g.add_node('TermienEast', x=800, y=880, z=0,
               comps={'building': 'SubstationepsNTU',})
                      # 'DHW': 'BuildingFixed'})
if n_neighs >= 9:
    g.add_node('TermienWest', x=0, y=0, z=0,
               comps={'building': 'SubstationepsNTU',})
                      # 'DHW': 'BuildingFixed'})

In [33]:
pipes = []

if n_neighs >= 1:
    g.add_edge('Producer', 'p1', name='dist_pipe0')
    g.add_edge('p1', 'WaterscheiGarden', name='dist_pipe2')
    pipes.append('dist_pipe0')
    pipes.append('dist_pipe2')
if n_neighs >= 2:
    g.add_edge('p1', 'ZwartbergNEast', name='dist_pipe1')
    pipes.append('dist_pipe1')
if n_neighs >= 3:
    g.add_edge('p1', 'p2', name='dist_pipe3')
    g.add_edge('p2', 'ZwartbergNWest', name='dist_pipe4')
    pipes.append('dist_pipe3')
    pipes.append('dist_pipe4')
if n_neighs >= 4:
    g.add_edge('p2', 'ZwartbergSouth', name='dist_pipe5')
    pipes.append('dist_pipe5')
if n_neighs >= 5:
    g.add_edge('p2', 'p3', name='dist_pipe6')
    g.add_edge('p3', 'OudWinterslag', name='dist_pipe7')
    pipes.append('dist_pipe6')
    pipes.append('dist_pipe7')
if n_neighs >= 6:
    g.add_edge('p3', 'p4', name='dist_pipe8')
    g.add_edge('p4', 'Winterslag', name='dist_pipe10')
    pipes.append('dist_pipe8')
    pipes.append('dist_pipe10')
if n_neighs >= 7:
    g.add_edge('p4', 'Boxbergheide', name='dist_pipe9')
    pipes.append('dist_pipe9')
if n_neighs >= 8:
    g.add_edge('p4', 'p5', name='dist_pipe11')
    g.add_edge('p5', 'TermienEast', name='dist_pipe12')
    pipes.append('dist_pipe11')
    pipes.append('dist_pipe12')
if n_neighs >= 9:
    g.add_edge('p5', 'TermienWest', name='dist_pipe13')
    pipes.append('dist_pipe13')

## The modesto object

In [34]:
optmodel = modesto.main.Modesto(pipe_model='FiniteVolumePipe', graph=g, temperature_driven=True)
optmodel.opt_settings(allow_flow_reversal=False)

## Load data

In [35]:
heat_profile = ut.read_time_data(resource_filename(
        'modesto', 'Data/HeatDemand'), name='TEASER_GenkNET_per_neighb.csv')

t_amb = ut.read_time_data(resource_filename('modesto', 'Data/Weather'), name='extT.csv')['Te']
t_g = pd.Series(12 + 273.15, index=t_amb.index)

wd = ut.read_time_data(resource_filename('modesto', 'Data'), name='Weather/weatherData.csv')
QsolN = wd['QsolN']
QsolE = wd['QsolE']
QsolS = wd['QsolS']
QsolW = wd['QsolW']

c_f = ut.read_time_data(path=resource_filename('modesto', 'Data'), name='ElectricityPrices/DAM_electricity_prices-2014_BE.csv')['price_BE']

mults = ut.read_file(resource_filename(
    'modesto', 'Data/HeatDemand'), name='TEASER_number_of_buildings.csv', timestamp=False).loc['Number of buildings']

## Parameters

In [36]:
general_params = {'Te': t_amb,
                  'Tg': t_g,
                  'Q_sol_E': QsolE,
                  'Q_sol_W': QsolW,
                  'Q_sol_S': QsolS,
                  'Q_sol_N': QsolN,
                  'time_step': time_step,
                  'horizon': horizon,
                  'elec_cost': c_f}

optmodel.change_params(general_params)

In [37]:
for n in range(n_neighs):
    neigh = neighs[n]
    building_params = {
        'mult': mults[neigh],
        'heat_flow': heat_profile[neigh] / mults[neigh],
        'temperature_radiator_in': 47 + 273.15,
        'temperature_radiator_out': 35 + 273.15,
        'temperature_supply_0': 60 + 273.15,
        'temperature_return_0': 40 + 273.15,
        'lines': ['supply', 'return'],
        'thermal_size_HEx': 15000,
        'exponential_HEx': 0.7,
        'mf_prim_0': 0.2
    }

    optmodel.change_params(building_params, node=neigh, comp='building')

In [38]:
for i, pipe in enumerate(pipes):
    pipe_params = {'diameter': diameters[all_pipes.index(pipe)],
                   'max_speed': 1.5,
                   'Courant': 1,
                   'Tg': pd.Series(12+273.15, index=t_amb.index),
                   'Tsup0': 57+273.15,
                   'Tret0': 40+273.15,
                   }

    optmodel.change_params(pipe_params, comp=pipe)

In [39]:
prod_design = {'efficiency': 1,
               'PEF': 1,
               'CO2': 0.178,  # based on HHV of CH4 (kg/KWh CH4)
               'fuel_cost': pd.Series([0.25] * int(n_steps/2) + [0.5] * (n_steps - int(n_steps/2))),
               # http://ec.europa.eu/eurostat/statistics-explained/index.php/Energy_price_statistics (euro/kWh CH4)
               'Qmax': 1.5e12,
               'ramp_cost': 0,
               'CO2_price': c_f,
               'temperature_max': 90 + 273.15,
               'temperature_min': 57 + 273.15,
               'temperature_supply_0': 65 + 273.15,
               'temperature_return_0': 30 + 273.15,
               'heat_estimate': heat_profile['WaterscheiGarden'] + heat_profile['ZwartbergNEast']}

optmodel.change_params(prod_design, 'Producer', 'plant')

## Compilation and solving

In [40]:
compile_order = [['Producer', None],
                 ['Producer', 'plant']]

if n_neighs >= 1:
    compile_order.insert(0, [None, 'dist_pipe0'])
    compile_order.insert(0, ['p1', None])
    compile_order.insert(0, [None, 'dist_pipe2'])
if n_neighs >= 2:
    compile_order.insert(0, [None, 'dist_pipe1'])
if n_neighs >= 3:
    compile_order.insert(0, [None, 'dist_pipe3'])
    compile_order.insert(0, ['p2', None])
    compile_order.insert(0, [None, 'dist_pipe4'])
if n_neighs >= 4:
    compile_order.insert(0, [None, 'dist_pipe5'])
if n_neighs >= 5:
    compile_order.insert(0, [None, 'dist_pipe6'])
    compile_order.insert(0, ['p3', None])
    compile_order.insert(0, [None, 'dist_pipe7'])
if n_neighs >= 6:
    compile_order.insert(0, [None, 'dist_pipe8'])
    compile_order.insert(0, ['p4', None])
    compile_order.insert(0, [None, 'dist_pipe10'])
if n_neighs >= 7:
    compile_order.insert(0, [None, 'dist_pipe9'])
if n_neighs >= 8:
    compile_order.insert(0, [None, 'dist_pipe11'])
    compile_order.insert(0, ['p5', None])
    compile_order.insert(0, [None, 'dist_pipe12'])
if n_neighs >= 9:
    compile_order.insert(0, [None, 'dist_pipe13'])
for n in range(n_neighs):
    compile_order.insert(0, [neighs[n], None])
    compile_order.insert(0, [neighs[n], 'building'])

In [41]:
optmodel.compile(start_time=start_time,
                 compile_order=compile_order)

optmodel.set_objective('cost')

optmodel.solve(tee=True, mipgap=0.2, last_results=False, g_describe=[480])

dist_pipe0 has 6 volumes, one element has a length of 310.01792062897124
dist_pipe2 has 3 volumes, one element has a length of 333.3333333333333
dist_pipe1 has 2 volumes, one element has a length of 316.22776601683796
Time to compile:  1.8198671340942383 

This is Ipopt version 3.12.3, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:    13023
Number of nonzeros in inequality constraint Jacobian.:      400
Number of nonzeros in Lagrangian Hessian.............:     5064

Total number of variables............................:     3401
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:     3301
Total number of inequality constraints...............:      400
        inequality cons

  78 1.5808391e+005 2.07e+003 1.95e+003  -1.0 2.64e+010    -  2.46e-003 3.40e-005h  9
  79 1.5810703e+005 2.07e+003 4.59e+003  -1.0 2.84e+010    -  1.01e-002 2.10e-005h 10
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  80r1.5810703e+005 2.07e+003 1.00e+003   2.2 0.00e+000    -  0.00e+000 2.86e-007R 17
  81r1.5810703e+005 2.05e+003 9.99e+002   2.2 1.57e+005    -  6.00e-003 1.00e-003f  1
  82 1.5451586e+005 2.05e+003 2.41e+001  -1.0 3.31e+012    -  5.78e-004 2.37e-005f  1
  83 1.5453129e+005 2.05e+003 1.32e+003  -1.0 1.53e+010    -  1.21e-002 2.28e-004h  1
  84 1.5467701e+005 2.05e+003 1.34e+003  -1.0 2.50e+010    -  3.19e-004 1.94e-004h  7
  85 1.5481419e+005 2.05e+003 1.37e+003  -1.0 2.55e+010    -  4.95e-004 1.95e-004h  7
  86 1.5496358e+005 2.05e+003 1.71e+003  -1.0 2.61e+010    -  2.97e-003 1.98e-004h  7
  87 1.5503108e+005 2.05e+003 1.72e+003  -1.0 2.67e+010    -  1.34e-004 9.57e-005h  8
  88 1.5509829e+005 2.05e+003 1.81e+003  -1.0 2.70e+010    

 168r1.5106755e+005 1.88e+003 1.00e+003   1.9 0.00e+000    -  0.00e+000 4.82e-007R 16
 169r1.5106755e+005 1.86e+003 3.26e+003   1.9 8.92e+004    -  2.71e-005 4.85e-004f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 170r1.5106755e+005 1.85e+003 3.26e+003   1.2 9.85e+002   0.0 6.27e-003 9.95e-004f  1
 171r1.5106755e+005 1.83e+003 3.35e+003   1.2 2.79e+003  -0.5 2.79e-002 1.70e-003f  1
 172r1.5106755e+005 1.64e+003 5.24e+003   1.2 1.03e+003  -0.1 1.71e-002 1.99e-002f  1
 173r1.5106755e+005 1.66e+003 7.93e+003   1.2 1.88e+003  -0.5 8.92e-003 1.01e-002f  1
 174r1.5106755e+005 2.62e+003 8.38e+003   1.2 5.86e+002  -0.1 3.23e-002 9.35e-003f  1
 175r1.5106755e+005 3.53e+003 4.46e+003   1.2 4.81e+002  -0.6 2.84e-002 3.48e-002f  1
 176r1.5106755e+005 3.44e+003 4.50e+004   1.2 2.77e+002  -0.2 5.83e-002 4.74e-002f  1
 177r1.5106755e+005 3.28e+003 1.65e+005   1.2 6.33e+001   1.2 6.26e-002 9.99e-002f  1
 178r1.5106755e+005 3.20e+003 7.25e+004   1.2 1.60e+001   2

 256 8.4647370e+004 2.08e+002 4.76e+008  -1.0 3.87e+010    -  3.17e-004 3.17e-004s 11
 257r8.4647370e+004 2.08e+002 1.00e+003   1.1 0.00e+000    -  0.00e+000 0.00e+000R  1
 258r8.4647346e+004 2.12e+002 1.00e+003   1.1 2.13e+004    -  1.61e-003 1.02e-003f  1
 259 8.4639097e+004 2.11e+002 2.19e+002  -1.0 5.93e+006    -  3.21e-002 4.98e-003f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 260 8.4638973e+004 2.11e+002 4.23e+004  -1.0 6.26e+006    -  6.74e-003 4.69e-005h  1
 261r8.4638973e+004 2.11e+002 9.99e+002   0.6 0.00e+000    -  0.00e+000 3.85e-007R  3
 262r8.4638935e+004 2.21e+002 9.98e+002   0.6 3.52e+004    -  5.19e-004 1.28e-003f  1
 263r8.4638926e+004 2.22e+002 9.98e+002   0.6 9.96e+003    -  3.19e-004 1.82e-003f  1
 264r8.4638924e+004 2.15e+002 9.97e+002   0.6 9.02e+003    -  9.33e-004 2.47e-003f  1
 265 8.4530758e+004 2.15e+002 1.06e+002  -1.0 3.43e+008    -  6.10e-004 2.33e-003f  1
 266 8.4530348e+004 2.15e+002 4.76e+004  -1.0 7.69e+007    

 343r8.3955084e+004 4.87e+002 9.95e+002  -0.1 1.01e+004    -  7.45e-004 1.05e-003f  1
 344r8.3955071e+004 5.33e+002 9.91e+002  -0.1 1.03e+004    -  1.46e-003 3.72e-003f  1
 345 8.3932509e+004 5.33e+002 2.02e+003  -1.0 2.43e+008    -  2.55e-003 7.65e-004f  1
 346 8.3938700e+004 5.32e+002 2.48e+005  -1.0 1.37e+008    -  3.89e-002 5.54e-004h  1
 347 8.3947773e+004 5.32e+002 4.70e+005  -1.0 1.62e+008    -  3.32e-002 7.06e-004h  3
 348 8.3953428e+004 5.32e+002 1.43e+006  -1.0 1.55e+008    -  1.14e-001 3.84e-004h  4
 349 8.3959246e+004 5.31e+002 3.17e+006  -1.0 3.16e+008    -  6.58e-002 3.57e-004h  2
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 350 8.3966246e+004 5.31e+002 5.27e+007  -1.0 1.13e+008    -  1.66e-001 5.16e-004h  4
 351 8.3974825e+004 5.31e+002 2.15e+009  -1.0 1.03e+008    -  2.26e-001 6.66e-004h  4
 352 8.3978317e+004 5.31e+002 2.96e+010  -1.0 1.27e+008    -  6.11e-002 2.94e-004h  5
 353 8.3981461e+004 5.31e+002 3.22e+011  -1.0 1.90e+008    

...

2569r9.2974948e+004 2.43e+003 1.39e+005   1.0 7.42e-007   9.3 1.00e+000 1.00e+000f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2570r9.2974948e+004 2.43e+003 3.20e+005   1.0 3.79e-007   9.7 1.00e+000 1.00e+000f  1
2571r9.2974948e+004 2.43e+003 2.84e+005   1.0 8.34e-007   9.2 1.00e+000 1.00e+000f  1
2572r9.2974948e+004 2.43e+003 7.32e+004   1.0 3.91e-008  10.6 1.00e+000 1.00e+000f  1
2573r9.2974948e+004 2.43e+003 7.30e+004   1.0 1.17e-007  10.1 1.00e+000 1.00e+000f  1
2574r9.2974948e+004 2.43e+003 6.58e+004   1.0 3.52e-007   9.6 1.00e+000 1.00e+000f  1
2575r9.2974948e+004 2.43e+003 2.33e+005   1.0 1.06e-006   9.1 1.00e+000 1.00e+000f  1
2576r9.2974948e+004 2.43e+003 7.25e+004   1.0 4.95e-008  10.5 1.00e+000 1.00e+000f  1
2577r9.2974948e+004 2.43e+003 8.69e+004   1.0 1.48e-007  10.0 1.00e+000 1.00e+000f  1
2578r9.2974948e+004 2.43e+003 1.65e+005   1.0 4.79e-007   9.5 1.00e+000 1.00e+000f  1
2579r9.2974948e+004 2.43e+003 2.80e+005   1.0 1.34e-006   9

2656r9.2974948e+004 2.43e+003 9.75e+004   1.0 8.60e-008  10.2 1.00e+000 1.00e+000f  1
2657r9.2974948e+004 2.43e+003 1.41e+005   1.0 2.58e-007   9.7 1.00e+000 1.00e+000f  1
2658r9.2974948e+004 2.43e+003 7.00e+004   1.0 9.67e-008  10.2 1.00e+000 1.00e+000f  1
2659r9.2974948e+004 2.43e+003 1.34e+005   1.0 2.90e-007   9.7 1.00e+000 1.00e+000f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2660r9.2974948e+004 2.43e+003 7.31e+004   1.0 1.09e-007  10.1 1.00e+000 1.00e+000f  1
2661r9.2974948e+004 2.43e+003 1.45e+005   1.0 3.27e-007   9.6 1.00e+000 1.00e+000f  1
2662r9.2974948e+004 2.43e+003 9.95e+004   1.0 1.22e-007  10.1 1.00e+000 1.00e+000f  1
2663r9.2974948e+004 2.43e+003 1.71e+005   1.0 3.67e-007   9.6 1.00e+000 1.00e+000f  1
2664r9.2974948e+004 2.43e+003 1.63e+005   1.0 1.48e-007  10.0 1.00e+000 1.00e+000f  1
2665r9.2974948e+004 2.43e+003 2.09e+005   1.0 4.13e-007   9.5 1.00e+000 1.00e+000f  1
2666r9.2974948e+004 2.43e+003 3.14e+005   1.0 2.09e-007  10

2758r9.2974948e+004 2.43e+003 1.07e+006   1.0 1.49e-006   9.4 1.00e+000 1.00e+000f  1
2759r9.2974948e+004 2.43e+003 3.21e+005   1.0 1.64e-006   8.9 1.00e+000 1.00e+000f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2760r9.2974948e+004 2.43e+003 2.28e+005   1.0 9.12e-008  10.3 1.00e+000 1.00e+000f  1
2761r9.2974948e+004 2.43e+003 1.83e+005   1.0 2.30e-007   9.8 1.00e+000 1.00e+000f  1
2762r9.2974948e+004 2.43e+003 9.68e+004   1.0 8.63e-008  10.2 1.00e+000 1.00e+000f  1
2763r9.2974948e+004 2.43e+003 1.40e+005   1.0 2.59e-007   9.7 1.00e+000 1.00e+000f  1
2764r9.2974948e+004 2.43e+003 6.93e+004   1.0 9.71e-008  10.2 1.00e+000 1.00e+000f  1
2765r9.2974948e+004 2.43e+003 1.33e+005   1.0 2.91e-007   9.7 1.00e+000 1.00e+000f  1
2766r9.2974948e+004 2.43e+003 7.23e+004   1.0 1.09e-007  10.1 1.00e+000 1.00e+000f  1
2767r9.2974948e+004 2.43e+003 1.45e+005   1.0 3.28e-007   9.6 1.00e+000 1.00e+000f  1
2768r9.2974948e+004 2.43e+003 9.84e+004   1.0 1.23e-007  10

2875r9.2974948e+004 2.43e+003 1.70e+005   1.0 3.70e-007   9.6 1.00e+000 1.00e+000f  1
2876r9.2974948e+004 2.43e+003 1.59e+005   1.0 1.47e-007  10.0 1.00e+000 1.00e+000f  1
2877r9.2974948e+004 2.43e+003 2.07e+005   1.0 4.17e-007   9.5 1.00e+000 1.00e+000f  1
2878r9.2974948e+004 2.43e+003 3.06e+005   1.0 2.08e-007  10.0 1.00e+000 1.00e+000f  1
2879r9.2974948e+004 2.43e+003 2.52e+005   1.0 4.69e-007   9.5 1.00e+000 1.00e+000f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2880r9.2974948e+004 2.43e+003 7.50e+005   1.0 3.76e-007   9.9 1.00e+000 1.00e+000f  1
2881r9.2974948e+004 2.43e+003 2.94e+005   1.0 5.27e-007   9.4 1.00e+000 1.00e+000f  1
2882r9.2974948e+004 2.43e+003 5.43e+006   1.0 2.11e-006   9.9 1.00e+000 1.00e+000f  1
2883r9.2974948e+004 2.43e+003 3.25e+005   1.0 5.93e-007   9.4 1.00e+000 1.00e+000f  1
2884r9.2974948e+004 2.43e+003 5.77e+004   1.0 2.78e-008  10.7 1.00e+000 1.00e+000f  1
2885r9.2974948e+004 2.43e+003 4.88e+004   1.0 8.34e-008  10

2971r9.2974948e+004 2.43e+003 3.20e+005   1.0 1.65e-006   8.9 1.00e+000 1.00e+000f  1
2972r9.2974948e+004 2.43e+003 2.26e+005   1.0 9.14e-008  10.3 1.00e+000 1.00e+000f  1
2973r9.2974948e+004 2.43e+003 1.83e+005   1.0 2.32e-007   9.8 1.00e+000 1.00e+000f  1
2974r9.2974948e+004 2.43e+003 9.56e+004   1.0 8.71e-008  10.2 1.00e+000 1.00e+000f  1
2975r9.2974948e+004 2.43e+003 1.40e+005   1.0 2.61e-007   9.7 1.00e+000 1.00e+000f  1
2976r9.2974948e+004 2.43e+003 6.82e+004   1.0 9.79e-008  10.2 1.00e+000 1.00e+000f  1
2977r9.2974948e+004 2.43e+003 1.32e+005   1.0 2.94e-007   9.7 1.00e+000 1.00e+000f  1
2978r9.2974948e+004 2.43e+003 7.10e+004   1.0 1.10e-007  10.1 1.00e+000 1.00e+000f  1
2979r9.2974948e+004 2.43e+003 1.44e+005   1.0 3.31e-007   9.6 1.00e+000 1.00e+000f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2980r9.2974948e+004 2.43e+003 9.65e+004   1.0 1.24e-007  10.1 1.00e+000 1.00e+000f  1
2981r9.2974948e+004 2.43e+003 1.69e+005   1.0 3.72e-007   9

## Collect results

In [ ]:
# Heat flows and mass flows
prod_hf = optmodel.get_result('heat_flow', node='Producer', comp='plant')
prod_mf = optmodel.get_result('mass_flow', node='Producer', comp='plant')
neigh_hf = pd.DataFrame(columns=[neighs[i] for i in range(n_neighs)])
neigh_mf = pd.DataFrame(columns=[neighs[i] for i in range(n_neighs)])

for n in range(n_neighs):
    neigh = neighs[n]
    mult = mults[neigh]
    neigh_hf[neigh] = (optmodel.get_result('heat_flow', node=neigh, comp='building')*mult)
    neigh_mf[neigh] = (optmodel.get_result('mf_prim', node=neigh, comp='building')*mult)

In [ ]:
# Temperatures
prod_T_sup = optmodel.get_result('Tsup', node='Producer', comp='plant') - 273.15
prod_T_ret = optmodel.get_result('Tret', node='Producer', comp='plant') - 273.15
neigh_T_sup = pd.DataFrame(columns=[neighs[i] for i in range(n_neighs)])
neigh_T_ret = pd.DataFrame(columns=[neighs[i] for i in range(n_neighs)])
slack = pd.DataFrame(columns=[neighs[i] for i in range(n_neighs)])

for n in range(n_neighs):
    neigh = neighs[n]
    mult = mults[neigh]
    neigh_T_sup[neigh] = (optmodel.get_result('Tpsup', node=neigh, comp='building') - 273.15)
    neigh_T_ret[neigh] = (optmodel.get_result('Tpret', node=neigh, comp='building') - 273.15)
    # slack[neigh] = optmodel.get_result('hf_slack', node=neigh, comp='building')

In [ ]:
prod_e = sum(prod_hf)
neigh_e = neigh_hf.sum(axis=0)

# Efficiency
print('\nNetwork')
print('Efficiency', (sum(neigh_e)) / (prod_e + 0.00001) * 100, '%')

In [ ]:
fig= plt.figure()
ax = fig.add_subplot(111)
ax.plot(prod_hf, label='Producer')
ax.plot(neigh_hf.sum(axis=1), label='Users and storage')
ax.axhline(y=0, linewidth=2, color='k', linestyle='--')
ax.set_title('Heat flows [W]')
ax.legend()
fig.tight_layout()

In [ ]:
fig1, axarr = plt.subplots(1, 1)
axarr.plot(prod_mf, label='Producer')
for neigh in [neighs[i] for i in range(n_neighs)]:
    axarr.plot(neigh_mf[neigh], label=neigh)
axarr.plot(neigh_mf.sum(axis=1), label='all users')
axarr.set_title('Mass flows network')
axarr.legend()
fig1.tight_layout()

In [ ]:
fig2, axarr = plt.subplots(1, 1)
axarr.plot(prod_T_sup, label='Producer Supply', color='r')
axarr.plot(prod_T_ret, label='Producer Return', linestyle='--', color='r')
for neigh in [neighs[i] for i in range(n_neighs)]:
    axarr.plot(neigh_T_sup[neigh], label='{} Supply'.format(neigh))
    axarr.plot(neigh_T_ret[neigh], label='{} Return'.format(neigh), linestyle='--')
axarr.legend()
axarr.set_title('Network temperatures')
fig2.tight_layout()